In [3]:
# MAIN BLOCK OF CODE
from datetime import * 
import pandas as pd
import openpyxl as op

# Reads excel file
df = pd.read_excel("RunLog.xlsx") 
user_action = input("Would you like to add data, edit data, delete data, or exit app? a/e/d/exit ")

# Add new row of data
def input_data():
    df = pd.read_excel("RunLog.xlsx") 
    add_data = "y"

    while add_data == "y":
        # User inputs with error handling
        date_str = input("Enter date (M/D/YYYY): ")
        try:
            # Convert date string into date object (has time)
            datetime_object = datetime.strptime(date_str, "%m/%d/%Y")

            # Extract just the date part to get a pure date object
            date = datetime_object.date()
        except ValueError:
            date_str = input("ERROR! Enter  valid date (M/D/YYYY): ")
            datetime_object = datetime.strptime(date_str, "%m/%d/%Y") 
            date = datetime_object.date()

        run_type = input("Enter run as \"road\" or \"trail\": ")
        if run_type != "road" or run_type != "trail":
            run_type = input("ERROR! Enter run as \"road\" or \"trail\": ")

        try:
            hours = int(input("Enter number of hours: "))
        except ValueError:
             hours = int(input("ERROR! Enter number of hours: "))

        try:
            minutes = int(input("Enter number of minutes: "))
        except ValueError:
            minutes = int(input("ERROR! Enter number of minutes: "))

        try:
            seconds = int(input("Enter number of seconds: "))
        except ValueError:
            seconds = int(input("ERROR! Enter number of seconds: "))

        try:
            miles = float(input("Enter number of miles: "))
        except ValueError:
             miles = float(input("ERROR! Enter number of miles: "))

        # Calculating pace
        convert_seconds = seconds / 60
        convert_hours = hours * 60
        total_minutes = minutes + convert_seconds + convert_hours
        pace = total_minutes / miles

        # Creating new row of data based on user inputs
        new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles], 'Pace':[pace]})

        # Adding new row 
        df = pd.concat([df, new_row], ignore_index=True)

        # Sends new row to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)

        # Asking if user wants to continue adding more data
        add_data = input("Would you like to input data? y/n ")
        if add_data == "y":
            continue 
        else: 
            break 
    display(df)

# Edit data
def edit_the_data():
    df = pd.read_excel("RunLog.xlsx") 
    edit_data = "y" 

    while edit_data == "y":
        # User inputting what row and column to make edits in 
        index_input = int(input("Enter index to edit: "))
        select_column = input("Enter column name to edit: ")
       
        # Depending on column, will convert user input to correct data type
        if select_column == "Date":
            user_edit = input("Enter new data: ")
            
            # Convert date string into date object (has time)
            datetime_object = datetime.strptime(user_edit, "%m/%d/%Y")

            # Extract just the date part to get a pure date object
            date = datetime_object.date()

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = date
        elif select_column == "Run Type":
            user_edit = input("Enter new data: ")

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit
        elif select_column == "Miles":
            df.at[index_input, 'Miles'] = None
            user_edit = float(input("Enter new data: "))

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit

            # Recalculating pace
            convert_seconds = df.loc[index_input].at["Seconds"] / 60
            convert_hours = df.loc[index_input].at["Hours"] * 60
            total_minutes = df.loc[index_input].at["Minutes"] + convert_seconds + convert_hours
            pace = total_minutes / df.loc[index_input].at["Miles"]

            df.at[index_input, "Pace"] = pace

        else:
            user_edit = int(input("Enter new data: "))

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit

            # Recalculating pace
            convert_seconds = df.loc[index_input].at["Seconds"] / 60
            convert_hours = df.loc[index_input].at["Hours"] * 60
            total_minutes = df.loc[index_input].at["Minutes"] + convert_seconds + convert_hours
            pace = total_minutes / df.loc[index_input].at["Miles"]

            df.at[index_input, "Pace"] = pace

        # Sends edit to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)
        
        # Asking if user wants to continue editing data
        edit_data = input("Would you like to edit more data? y/n ")
        if edit_data == "y":
            continue
        else:
            break
    display(df)

# Delete row
def delete_row():
    df = pd.read_excel("RunLog.xlsx") 
    delete_data = "y"

    while delete_data == "y":
        # User inputting what row to delete
        index_input = int(input("Enter index to delete: "))

        # Delete row base on index
        df.drop([index_input], inplace=True)

        # Sends edit to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)
        
        # Asking if user wants to continue to delete more rows of data
        delete_data = input("Would you like to delete another row? y/n ")
        if delete_data == "y": 
            continue
        else:
            break
    display(df)

# Base on user's choice, will either add, edit, delete, or show the table if exiting app
if user_action == "a":
    input_data()
if user_action == "e":
    edit_the_data()
if user_action == "d":
    delete_row()
if user_action == "exit":
    display(df)




In [2]:
# MAIN CODE
# Temporary utilizing a different excel spreadsheet with a whole months worth of inputted running data

import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx") 
df.set_index('Date', inplace=True)

# Function converts the time values in the DataFrame to total hours and calculates the pace
def convert_to_TotalHours(df):
    # Make a new column for total time in hours by adding hours, minutes, and seconds as timedeltas    
    df['TotalHours'] = (pd.to_timedelta(df['Hours'], unit='h') + 
                pd.to_timedelta(df['Minutes'], unit='m') + 
                pd.to_timedelta(df['Seconds'], unit='s')).dt.total_seconds() / 3600

    # Drop the Hours, Minutes, and Seconds columns as they are no longer needed            
    columns_to_drop = ['Hours', 'Minutes', 'Seconds']
    df_copy = df.copy() # needed otherwise get a warning    
    df_copy.drop(columns=columns_to_drop, inplace=True)
    df = df_copy

    # Add a Pace column
    df['Pace'] = df['TotalHours'] / df['Miles']
    return df

# Function that takes a df, a month as an integer and a runtype and returns the daily average miles per month
def get_monthly_averages(df, month, runtype):
    df_month = df[df.index.month == month]
    if len(df_month) == 0:
        return None, None, None # need 3 None's to match the return type
    runtype_group = df_month.groupby('Run Type')   
    monthly = runtype_group.resample('ME')
    monthly_mean = monthly.mean() 
    row = monthly_mean.loc[runtype]
    return float(row['Miles'].iloc[0]), float(row['TotalHours'].iloc[0]), float(row['Pace'].iloc[0])


df = convert_to_TotalHours(df)
for month in range(1, 13):
    for runtype in ['road', 'trail']:
        miles, total_hours, pace = get_monthly_averages(df, month, runtype)
        if miles is not None: # all three are None if there is no data for that month
            print(f"Daily average {runtype} miles for Month {month}: {miles} miles")
            print()  # print a newline 




Daily average road miles for Month 1: 2.621875 miles

Daily average trail miles for Month 1: 3.5266666666666664 miles

Daily average road miles for Month 2: 12.0 miles

Daily average trail miles for Month 2: 11.0 miles



In [11]:
# NOTE: OLD
# Temporary utilizing a different excel spreadsheet with a whole months worth of inputted running data

import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx")
df.set_index('Date', inplace=True)
# df['Date']=pd.to_datetime(df['Date'])

road_miles = 0.0
trail_miles = 0.0

month_input = input("Enter month: ")
# df = df[df["Date"].dt.month == month_input]
# df = df[(df['Date']).dt.strftime('%B') == month_input]
# df=df[((df['Date'].dt.month==month_input))]
# df[df['Dates'].month == 11]['Dates']

runtype_group = df.groupby('Run Type')
monthly = runtype_group.resample('ME')   # CH was ME but need to be M for monthly
# print(monthly)

# FIXME: Add a way to interate base on month
# Iterates through each row and adds the total road and trail run miles
for index, row in df.iterrows():
    # while index == (df.loc[(df['Date'].dt.month==month_input)]):
    if df.at[index, "Run Type"] == "road":
        road_miles += df.at[index, "Miles"]
    if df.at[index, "Run Type"] == "trail":
        trail_miles += df.at[index, "Miles"]

# Calculations
weekly_road_average = road_miles / 4
weekly_trail_average = trail_miles / 4
weekly_total_average = weekly_road_average + weekly_trail_average # Weekly average for the month

print("Weekly Road Run Average:", weekly_road_average)
print("Weekly Trail Run Average:", weekly_trail_average)
print(f"Weekly Total Average for {month_input}: {weekly_total_average}")

Weekly Road Run Average: 13.4875
Weekly Trail Run Average: 18.725
Weekly Total Average for January: 32.212500000000006


In [36]:
################################################
# CH jun 27 copy of above for monthly averages #
################################################ 
import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx") # CH I took out the index column
df.set_index('Date', inplace=True)

In [37]:

def convert_to_TotalHours(df):
    """Converts the time values in the DataFrame to total hours and calculates the pace.
    Args:
        df (DataFrame): The input DataFrame containing columns 'Hours', 'Minutes', 'Seconds', and 'Miles'
        that are used to calculate 'TotalHours'. Pace is calculated as 'TotalHours' / 'Miles'.
    Returns:
        DataFrame: The modified DataFrame with dropped Hours, Minutes, Seconds and 
                   additional columns 'TotalHours' and 'Pace'.
    """
    import pandas as pd

    # Make a new column for total time in hours by adding hours, minutes, and seconds as timedeltas    
    df['TotalHours'] = (pd.to_timedelta(df['Hours'], unit='h') + 
                pd.to_timedelta(df['Minutes'], unit='m') + 
                pd.to_timedelta(df['Seconds'], unit='s')).dt.total_seconds() / 3600

    # drop the Hours, Minutes, and Seconds columns as they are no longer needed            
    columns_to_drop = ['Hours', 'Minutes', 'Seconds']
    df_copy = df.copy() # needed otherwise get a warning    
    df_copy.drop(columns=columns_to_drop, inplace=True)
    df = df_copy

    # add a Pace column
    df['Pace'] = df['TotalHours'] / df['Miles']
    return df

df = convert_to_TotalHours(df)
print(df)

           Run Type  Miles  TotalHours      Pace
Date                                            
2024-01-01     road   1.00    1.016944  1.016944
2024-01-02    trail   2.00    2.033889  1.016944
2024-01-03     road   1.00    1.016944  1.016944
2024-01-04    trail   2.00    2.033889  1.016944
2024-01-05     road   1.00    1.016944  1.016944
2024-01-06    trail   2.00    2.033889  1.016944
2024-01-07     road   2.00    2.033889  1.016944
2024-01-08    trail   3.00    3.058333  1.019444
2024-01-09     road   1.00    1.016944  1.016944
2024-01-10    trail   4.00    4.711111  1.177778
2024-01-11     road   3.00    3.050833  1.016944
2024-01-12    trail   2.50    2.038889  0.815556
2024-01-13     road   3.20    4.055556  1.267361
2024-01-14    trail   4.50    3.504167  0.778704
2024-01-15     road   2.00    2.669444  1.334722
2024-01-16    trail   2.50    3.252222  1.300889
2024-01-17     road   1.50    2.172222  1.448148
2024-01-18    trail   5.20    4.420000  0.850000
2024-01-19     road 

In [38]:
# Use case 1: just get averages for a month
month_input = input("Enter month (1 to 12):")
# Convert month_input to integer
month = int(month_input)

# Filter rows in df based on the month
df_month = df[df.index.month == month]
print(df_month)

           Run Type  Miles  TotalHours      Pace
Date                                            
2024-01-01     road   1.00    1.016944  1.016944
2024-01-02    trail   2.00    2.033889  1.016944
2024-01-03     road   1.00    1.016944  1.016944
2024-01-04    trail   2.00    2.033889  1.016944
2024-01-05     road   1.00    1.016944  1.016944
2024-01-06    trail   2.00    2.033889  1.016944
2024-01-07     road   2.00    2.033889  1.016944
2024-01-08    trail   3.00    3.058333  1.019444
2024-01-09     road   1.00    1.016944  1.016944
2024-01-10    trail   4.00    4.711111  1.177778
2024-01-11     road   3.00    3.050833  1.016944
2024-01-12    trail   2.50    2.038889  0.815556
2024-01-13     road   3.20    4.055556  1.267361
2024-01-14    trail   4.50    3.504167  0.778704
2024-01-15     road   2.00    2.669444  1.334722
2024-01-16    trail   2.50    3.252222  1.300889
2024-01-17     road   1.50    2.172222  1.448148
2024-01-18    trail   5.20    4.420000  0.850000
2024-01-19     road 

In [39]:
# group by runtype and resample by month
runtype_group = df_month.groupby('Run Type')   
monthly = runtype_group.resample('ME')
monthly_mean = monthly.mean() 
print(monthly_mean)  # I don't know why the date has been aggregated to the end of month ...


                        Miles  TotalHours      Pace
Run Type Date                                      
road     2024-01-31  2.621875    2.446233  1.003782
trail    2024-01-31  3.526667    2.911278  0.872369


In [40]:
print(monthly.min())
print(monthly.max())

                    Run Type  Miles  TotalHours      Pace
Run Type Date                                            
road     2024-01-31     road    1.0    1.016944  0.635590
trail    2024-01-31    trail    2.0    1.169444  0.578826
                    Run Type  Miles  TotalHours      Pace
Run Type Date                                            
road     2024-01-31     road    6.5    5.000000  1.448148
trail    2024-01-31    trail    5.8    4.711111  1.300889


In [41]:
# B/c the data is now grouped by runtype, road and trail are the index for the rows
for runtype in ['road', 'trail']:  
    row = monthly_mean.loc[runtype]
    print(f"Monthly {runtype} average for month {month} is:")
    #print(row)
    for k in ['Miles', 'TotalHours', 'Pace']:
        print(f"{k}: {float(row[k].iloc[0])}", end=' ')
    print()  # print a newline

# CH: I know that having to use float(row[k].iloc[0]) just to get a number is silly ... but I don't know how to avoid it   

Monthly road average for month 1 is:
Miles: 2.621875 TotalHours: 2.4462326388888886 Pace: 1.0037816798867853 
Monthly trail average for month 1 is:
Miles: 3.5266666666666664 TotalHours: 2.911277777777778 Pace: 0.872368987204511 


In [42]:
# function that takes a df, a month as an integer and a runtype and returns the average miles, total hours, and pace for it
def get_monthly_averages(df, month, runtype):
    """Calculate the monthly averages for a specific month and run type. If there are no entries for the specified month and run type, return None for all three values.

    Parameters:
    - df (DataFrame): The input DataFrame containing the running log data.
    - month (int): The month for which to calculate the averages (1-12).
    - runtype (str): The type of run for which to calculate the averages.

    Returns:
    - miles (float): The average number of miles for the specified month and run type.
    - total_hours (float): The average total hours for the specified month and run type.
    - pace (float): The average pace for the specified month and run type.
    """
    df_month = df[df.index.month == month]
    if len(df_month) == 0:
        return None, None, None # need 3 None's to match the return type
    runtype_group = df_month.groupby('Run Type')   
    monthly = runtype_group.resample('ME')
    monthly_mean = monthly.mean() 
    row = monthly_mean.loc[runtype]
    return float(row['Miles'].iloc[0]), float(row['TotalHours'].iloc[0]), float(row['Pace'].iloc[0])



In [43]:
df = pd.read_excel("RunLog_Test.xlsx") # CH I took out the index column
df.set_index('Date', inplace=True)
df = convert_to_TotalHours(df)
for month in range(1, 13):
    for runtype in ['road', 'trail']:
        miles, total_hours, pace = get_monthly_averages(df, month, runtype)
        if miles is not None: # all three are None if there is no data for that month
            print(f"Monthly {runtype} average for month {month} is:")
            print(f"Miles: {miles}", end=' ')
            print(f"Total Hours: {total_hours}", end=' ' )
            print(f"Pace: {pace}")
            print()  # print a newline

Monthly road average for month 1 is:
Miles: 2.621875 Total Hours: 2.4462326388888886 Pace: 1.0037816798867853

Monthly trail average for month 1 is:
Miles: 3.5266666666666664 Total Hours: 2.911277777777778 Pace: 0.872368987204511

Monthly road average for month 2 is:
Miles: 12.0 Total Hours: 3.0508333333333333 Pace: 0.2542361111111111

Monthly trail average for month 2 is:
Miles: 11.0 Total Hours: 4.57625 Pace: 0.41525231481481484



In [ ]:
# NOTE: OLD CODE
# ADDING NEW DATA
# FIXME: Stop having new index columns added 
# FIXED: df.to_excel("RunLog.xlsx", index=False)

import pandas as pd
# import openpyxl as op

# Reads excel file
df = pd.read_excel("RunLog.xlsx") 

add_data = input("Would you like to input data? y/n ")

while add_data == "y":
    # User inputs
    date = input("Enter date (M/D/YYYY): ")
    run_type = input("Enter road or trail: ")
    hours = int(input("Enter number of hours: "))
    minutes = int(input("Enter number of minutes: "))
    seconds = int(input("Enter number of seconds: "))
    miles = float(input("Enter number of miles: "))

    # Creating new row of data based on user inputs
    new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles]})
    
    # Adding new row 
    df = pd.concat([df, new_row], ignore_index=True)
   
    # Sends new row to excel spreadsheet
    df.to_excel("RunLog.xlsx", index=False)

    # Asking if user wants to continue adding more data
    add_data = input("Would you like to input data? y/n ")
    if add_data == "y":
        continue 
    else: 
        break 
print(df)

In [ ]:
# CH with using datetime for dates
from datetime import datetime
import pandas as pd
import openpyxl as op

# Reads excel file
df = pd.read_excel("CH.xlsx") 

add_data = input("Would you like to input data? y/n ")

while add_data == "y":
    # User inputs
    date_str = input("Enter date (M/D/YYYY): ")
    run_type = input("Enter road or trail: ")
    hours = int(input("Enter number of hours: "))
    minutes = int(input("Enter number of minutes: "))
    seconds = int(input("Enter number of seconds: "))
    miles = float(input("Enter number of miles: "))

    # convert date string into date object (has time)
    datetime_object = datetime.strptime(date_str, "%m/%d/%Y")

    # Extract just the date part to get a pure date object
    date = datetime_object.date()

    # Creating new row of data based on user inputs
    new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles]})
    
    print(new_row)

    # Adding new row 
    df = pd.concat([df, new_row], ignore_index=True)
   
    # Sends new row to excel spreadsheet
    df.to_excel("CH.xlsx", index=False)

    # Asking if user wants to continue adding more data
    add_data = input("Would you like to input data? y/n ")
    if add_data == "y":
        continue 
    else: 
        break 
print(df)

In [ ]:
display(pd.read_excel("CH.xlsx")) 

In [ ]:
# NOTE: OLD EDITING DATA 
# FIXME: Stop having new index columns added 
# NOTE: HAS BEEN FIXED 

import pandas as pd

df = pd.read_excel("RunLog.xlsx") 
edit_data = input("Would you like to edit data? y/n ")

while edit_data == "y":
    # User inputting what row and column to make edits in
    index_input = int(input("Enter index to edit: "))
    select_column = input("Enter column name to edit: ")
    
    # Depending on column, will convert user input to correct data type
    if select_column == "Date" or select_column == "Run Type":
        user_edit = input("Enter new data: ")
    elif select_column == "Miles":
        df.at[index_input, 'Miles'] = None
        user_edit = float(input("Enter new data: "))
    else:
        user_edit = int(input("Enter new data: "))
     
    # Changes the data at the selected index and column
    df.at[index_input, select_column] = user_edit

    # Sends edit to excel spreadsheet
    df.to_excel("RunLog.xlsx", index=False)
    
    # Asking if user wants to continue editing data
    edit_data = input("Would you like to edit data? y/n ")
    if edit_data == "y":
        continue
    else:
        break
print(df)


In [ ]:
# NOTE: OLD CALCULATIONS 
# For now, utilizing a different excel spreadsheet with a whole months worth of inputted running data

import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx") 

road_miles = 0.0
trail_miles = 0.0
counter = 0

# Iterates through each row and adds the total road and trail run miles
for index, row in df.iterrows():
    if counter <= 27:
        if df.at[index, "Run Type"] == "road":
            road_miles += df.at[index, "Miles"]
            counter += 1
        if df.at[index, "Run Type"] == "trail":
            trail_miles += df.at[index, "Miles"]
            counter += 1
    elif counter > 28:
        counter = 0


# Calculations
weekly_road_average = road_miles / 4
weekly_trail_average = trail_miles / 4
weekly_total_average = weekly_road_average + weekly_trail_average

print("Weekly Road Run Average:", weekly_road_average)
print("Weekly Trail Run Average:", weekly_trail_average)
print("Weekly Total Average:", weekly_total_average)



In [ ]:
# CH
df = pd.read_excel("log_for_avg.xlsx") 
# df = pd.read_excel("RunLog_Test.xlsx") 

# Set 'Date' as the index
df.set_index('Date', inplace=True)
display(df)

runtype_group = df.groupby('Run Type')
weekly = runtype_group.resample('W')
weekly_miles_mean = weekly.mean()['Miles']

# sort (re-index) by date
weekly_miles_mean = weekly_miles_mean.sort_index(level='Date')

print(weekly)
print(weekly_miles_mean) # dataframe indexed by time



In [ ]:
# dropna() remove NaN rows
table_as_nested_list = list(weekly_miles_mean.dropna().items())
# items() give us a nested structure
table_as_nested_list

In [ ]:
# unpack the type, date and miles and print
for (run_type, timestamp), miles in table_as_nested_list:
    date_only = timestamp.date()
    print(f"{date_only}: {run_type}: {miles} on avg.")